Traditional processing

In [8]:
import os
from math import sqrt
from statistics import mean, median, stdev

First we'll load the data

In [2]:

data_files_list = ["./export/"+f for f in os.listdir("./export") ]
instance_dict = {}
instance_additionals = {}
set_sizes_dict = {}
for dir_str in data_files_list:
    with open(dir_str, 'r') as text_file:
        cnt = 0
        instance = ""
        for line in text_file:
            if cnt < 9:
                if cnt == 0:
                    instance = line.split()[0]
                    instance_dict[instance] = []
                if cnt == 4:
                    instance_additionals[instance] = []
                    split_line = line.split()
                    instance_additionals[instance].append([int(i) for i in split_line])
                cnt += 1
                continue
            split_line = line.split()
            instance_dict[instance].append([int(i) for i in split_line])
        text_file.close()

ng_dict = {}
cnt = -1
with open("ng_outs.csv", 'r') as text_file:
    for line in text_file:
        if cnt < 2:
            cnt += 1
            continue
        raw_line = line.strip()
        split_line_list = raw_line.split(sep=";")
        instance = split_line_list[3]
        if instance not in ng_dict:
            # ng_dict[instance] = [[0 for i in range(101)]]
            ng_dict[instance] = []
        ng_dict[instance].append([int(i) for i in split_line_list[5:-1]])
        set_sizes_dict[instance] = sum(sum(i) for i in ng_dict[instance])
        if len(split_line_list[5:-1]) != 100:
            print("case found for instance "+instance)
    text_file.close()

In [3]:
ordered_difficulty = [(k,v) for k,v in set_sizes_dict.items()]
ordered_difficulty = sorted(ordered_difficulty, key=lambda tup: tup[1], reverse=True)
instances_to_add = [ordered_difficulty[i][0] for i in range(0,4000)]
# instances_remaining = [ordered_difficulty[i][0] for i in range(1000,len(ordered_difficulty))]
# easy_instances = sample(instances_remaining, 2000)
instance_list = instances_to_add #+ easy_instances

In [4]:
data_files_list = ["./export2/"+f for f in os.listdir("./export2") ]

for dir_str in data_files_list:
    with open(dir_str, 'r') as text_file:
        cnt = 0
        instance = ""
        for line in text_file:
            if cnt < 9:
                if cnt == 0:
                    instance = line.split()[0]
                    instance_dict[instance] = []
                if cnt == 4:
                    instance_additionals[instance] = []
                    split_line = line.split()
                    instance_additionals[instance].append([int(i) for i in split_line])
                cnt += 1
                continue
            split_line = line.split()
            instance_dict[instance].append([int(i) for i in split_line])
        text_file.close()

cnt = -1
with open("ng_outs_JLA.csv", 'r') as text_file:
    for line in text_file:
        if cnt < 1:
            cnt += 1
            continue
        raw_line = line.strip()
        split_line_list = raw_line.split(sep=";")
        instance = split_line_list[3]
        if instance not in ng_dict:
            # ng_dict[instance] = [[0 for i in range(101)]]
            ng_dict[instance] = []
        ng_dict[instance].append([int(i) for i in split_line_list[5:]])
        set_sizes_dict[instance] = sum(sum(i) for i in ng_dict[instance])
        # add new instances
        instance_list.append(instance)
        if len(split_line_list[5:]) != 100:
            print("case found for instance "+instance)
    text_file.close()

In [5]:

data_files_list = ["./export8/"+f for f in os.listdir("./export8") ]
for dir_str in data_files_list:
    with open(dir_str, 'r') as text_file:
        cnt = 0
        instance = ""
        for line in text_file:
            if cnt < 9:
                if cnt == 0:
                    instance = line.split()[0]
                    instance_dict[instance] = []
                if cnt == 4:
                    instance_additionals[instance] = []
                    split_line = line.split()
                    instance_additionals[instance].append([int(i) for i in split_line])
                cnt += 1
                continue
            split_line = line.split()
            instance_dict[instance].append([int(i) for i in split_line])
        text_file.close()

cnt = -1
with open("ng_outs_29-11.csv", 'r') as text_file:
    for line in text_file:
        if cnt < 2:
            cnt += 1
            continue
        raw_line = line.strip()
        split_line_list = raw_line.split(sep=";")
        instance = split_line_list[3]
        if instance not in ng_dict:
            # ng_dict[instance] = [[0 for i in range(101)]]
            ng_dict[instance] = []
        ng_dict[instance].append([int(i) for i in split_line_list[5:-1]])
        set_sizes_dict[instance] = sum(sum(i) for i in ng_dict[instance])
        if instance in instance_dict.keys():
            instance_list.append(instance)
        if len(split_line_list[5:-1]) != 100:
            print("case found for instance "+instance)
    text_file.close()

now we'll process each graph in order to get all the desired variables

In [7]:
def distance(x1, y1, x2, y2):
    x = max(x1 - x2, x2 - x1)
    y = max(y1 - y2, y2 - y1)
    return sqrt(x**2 + y**2)


In [ ]:
data_dict = {}
for instance_name in instance_list:
    data_dict[instance_name] = {
        "nodes" : [],
        "graph" : {
            "vehicles": -1,
            "capacity": -1,
            "extension": -1,
            "avg_ext": -1,
            "avg_route_len": -1
            },
        "edges" : []
    }
    instance = instance_dict[instance_name]
    max_demand = -1
    total_demand = 0
    vehicles, capacity = instance_additionals[instance_name][0]
    data_dict[instance_name]["graph"]["vehicles"] = vehicles
    data_dict[instance_name]["graph"]["capacity"] = capacity
    node_d, xcoor_d, ycoor_d, _, ai_d, bi_d, _ = instance[0]
    dist_list = []
    for i in instance[1:]:
        demand = i[3]
        total_demand += demand
        if demand > max_demand:
            max_demand = demand
    data_dict[instance_name]["graph"]["avg_route_len"] = total_demand/capacity
    for i in instance[1:]:
        node, xcoor, ycoor, demand, ai, bi, st = i
        x_coor_norm = xcoor - xcoor_d
        y_coor_norm = ycoor - ycoor_d
        radial_dist = sqrt(x_coor_norm**2 + y_coor_norm**2)
        dist_list.append(radial_dist)
        data_dict[instance_name]["nodes"].append({
            "id": node,
            "demand": demand,
            "ai": ai,
            "bi": bi,
            "x": xcoor,
            "y": ycoor,
            "xn": x_coor_norm,
            "yn": y_coor_norm,
            "st": st,
            "rad": radial_dist
        })
    data_dict[instance_name]["graph"]["extension"] = max(dist_list)
    data_dict[instance_name]["graph"]["avg_ext"] = mean(dist_list)
    for i in range(100):
        for j in range(100):
            if i != j:
                i_nod = data_dict[instance_name]["nodes"][i]
                j_nod = data_dict[instance_name]["nodes"][j]
                dist = distance(i_nod["x"], i_nod["y"], j_nod["x"], j_nod["y"])
                alpha = max(i_nod["ai"] + i_nod["st"] + dist, j_nod["ai"])
                return_t_measure = max(0, j_nod["bi"] -  alpha)
                return_c_measure = capacity - (i_nod["demand"] + j_nod["demand"])
                data_dict[instance_name]["edges"].append({
                    "from": 
                })


        

        